In [1]:
from datetime import timedelta

from cadCAD import configs
from cadCAD.configuration import Configuration
from cadCAD.configuration.utils import exo_update_per_ts, bound_norm_random, ep_time_step

#define number of experiments N, and the length of each experiment with T
sim_config = {
    'N': 1,
    'T': range(100)
}

#this variable defines the time period assumed for each period in T
ts_format = '%Y-%m-%d %H:%M:%S'
t_delta = timedelta(days=0, minutes=1, seconds=0)
def time_model(step, sL, s, _input):
    y = 'timestamp'
    x = ep_time_step(s, dt_str=s['timestamp'], fromat_str=ts_format, _timedelta=t_delta)
    return (y, x)

#used with stochastic processes to make experiments repeatable
seed = {}

# Initial Conditions
genesis_states = {
    'Verifiers_On': True, #verifiers are active
    'Cheaters_On': False, #cheaters are inactive
    'Total_Volume': 100, #volume of transaction activity
    'Honest_Volume': 100, #volume of honest transaction activity
    'Cheats_Volume': 0, #volume of cheating transaction activity
    'Cheats_Caught_Volume': 0, #volume of cheating that was caught
    'Verifiers_Cost': 0, #cost incurred by verifiers
    'Verifiers_Reward': 0, #rewards collected by verifiers
    'Cheaters_Cost': 0, #costs incurred by cheaters
    'Cheater_Reward': 0, #rewards (profit) achieved by cheating 
    'timestamp': '2018-01-01 00:00:00'
}

# Verifier's cost per transaction verified
alfa = 0.001
def verifier_cost(s):
    return alfa * (s['Total_Volume'])

# Verifier's reward per cheat caught
beta = 4 
def verifier_reward(s):
    return beta * s['Cheats_Volume']

# Cheater's reward per transaction sent successfully
gamma = 1
def cheater_reward(s):
    return gamma * (s['Cheats_Volume'])

# Cheater's cost per cheat caught
delta = 5
def cheater_cost(s):
    return delta * s['Cheats_Caught_Volume']

# Behaviors

# verifiers required expected profit threshold before verifying
# this is a simple ROI threshold policy
theta = .1
def verifier(step, sL, s):
    act = False
    if (verifier_expected_reward(s) > (1+theta)*verifier_cost(s)):
        act = True
    return {'verifier': act}

#in order to implement the ROI heuristic for verifiers we need an estimator for Expected Rewards
def verifier_expected_reward(s):
    '''
    We assume the existence of an off-chain signaling mechanism 
    by which potential verifiers become aware of some of cheating volume.
    The gain of this signal is small because the assumption is it hearsay
    '''
    off_chain_cheating_signal = 0.01 #hearsay
    off_chain_expected_cheating = off_chain_cheating_signal * s['Cheats_Volume']
    on_chain_expected_cheating = s['Cheats_Caught_Volume']
    return beta * max([off_chain_expected_cheating, on_chain_expected_cheating])

# cheater's ROI threshold collapses to simply
# cheat if there are no verifiers
# don't cheat if there are verifiers
def cheater(step, sL, s):
    act = not(s['Verifiers_On'])
    return {'cheater': act}

#direct handling of actions to set state variables
def commit_resources_to_verifying(step, sL, s, _input):
    y = 'Verifiers_On'
    x = _input['verifier']
    return (y, x)

def commit_resources_to_cheating(step, sL, s, _input):
    y = 'Cheaters_On'
    x = _input['cheater']
    return (y, x)

# these mechanisms are for state updates are computed from other states
# due to the simplicity of this model; 
# exogenous states, and exogenous process features of the simulation framework are used
# these models can also be handled as standard states and mechanism 
# with only minor changes to the config

#the total volume is modeled as static but changing epsilon to a random variable
#is a simple way to introduce a stochastic process to make Monte Carlo simulations interesting
epsilon = 1
def volume_ep(step, sL, s, _input):
    y = 'Total_Volume'
    x = epsilon*s['Total_Volume']
    return (y, x)

#zeta is the percentage of the total volume which is cheating volume
#extend this model by making zeta an output of a Policy or a random variable
zeta=0.2
def cheat_volume_ep(step, sL, s, _input):
    y = 'Cheats_Volume'
    if (s['Cheaters_On']):
        x = zeta*(s['Total_Volume'])
    else:
        x = 0
    return (y, x)

#honest volume is the complement of the cheat volume
def honest_volume_ep(step, sL, s, _input):
    y = 'Honest_Volume'
    if (s['Cheaters_On']):
        x = (1-zeta)*s['Total_Volume']
    else:
        x = s['Total_Volume']
    return (y, x)

#resolve the cheat volume caught being caught
def cheats_caught_ep(step, sL, s, _input):
    y = 'Cheats_Caught_Volume'
    if (s['Verifiers_On']):
        x = s['Cheats_Volume']
    else:
        x = 0
    return (y, x)

#resolve the costs incurred by verifiers verifying transactions
def verifier_cost_ep(step, sL, s, _input):
    y = 'Verifiers_Cost'
    if (s['Verifiers_On']):
        x = verifier_cost(s)
    else:
        x = 0
    return (y, x)

#resolve the rewards earned by verifiers verifying transactions
def verifier_reward_ep(step, sL, s, _input):
    y = 'Verifiers_Reward'
    if (s['Verifiers_On']):
        x = verifier_reward(s)
    else:
        x = 0
    return (y, x)

#resolve the costs incurred by cheaters getting caught
def cheater_cost_ep(step, sL, s, _input):
    y = 'Cheaters_Cost'
    if (s['Verifiers_On']):
        x = cheater_cost(s)
    else:
        x = 0
    return (y, x)

#resolve the rewards won by cheaters getting away with cheating
def cheater_reward_ep(step, sL, s, _input):
    y = 'Cheater_Reward'
    if (s['Cheaters_On']):
        x = cheater_reward(s)
    else:
        x = 0
    return (y, x)


# store the state and state update functions 
#as "exogenous_states" updated as "ep" = "exogenous process"
# note this terminology is being deprecated 
# as it confuses internal dynamics with models of processes
# which occur outside of but have influence on our system
exogenous_states = exo_update_per_ts(
    {
    'Total_Volume': volume_ep,
    'Honest_Volume': honest_volume_ep,
    'Cheats_Volume': cheat_volume_ep,
    'Cheats_Caught_Volume': cheats_caught_ep,
    'Verifiers_Cost': verifier_cost_ep,
    'Verifiers_Reward': verifier_reward_ep,
    'Cheaters_Cost': cheater_cost_ep,
    'Cheater_Reward': cheater_reward_ep,
    'timestamp': time_model
    }
)

#not used in this simulation
env_processes = {
}

#wire together the partial state update blocks ~ see Robot and Marbles tutorials for more details
mechanisms = {
    'commit': {
        'behaviors': {
            'verifier': verifier,
            'cheater': cheater
        },
        'states': { 
            'Verifiers_On': commit_resources_to_verifying,
            'Cheaters_On': commit_resources_to_cheating            
        }
    }
}

configs.append(
    Configuration(
        sim_config=sim_config,
        state_dict=genesis_states,
        seed=seed,
        exogenous_states=exogenous_states,
        env_processes=env_processes,
        mechanisms=mechanisms
    )
)

from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
# from demos import simple_tracker_config
from cadCAD import configs
exec_mode = ExecutionMode()

single_config = [configs[0]]
single_proc_ctx = ExecutionContext(exec_mode.single_proc)
run = Executor(single_proc_ctx, single_config)
run_raw_result = run.main()[0]

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate
result = pd.DataFrame(run_raw_result)

result.head(20)

TypeError: ep_decorator() missing 5 required positional arguments: 'var_dict', 'sub_step', 'sL', 's', and '_input'